<a href="https://colab.research.google.com/github/louispaulet/Data_dependencies_and_data_integration_course/blob/main/DDDI_Project_IND.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import io

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/USEPA/FDTool/master/data/input/Table1.csv')
df


,A,B,C,D,E
0,0,0,0,2,0
1,0,1,0,2,0
2,0,2,0,2,2
3,0,3,1,2,0
4,4,1,1,1,4
5,4,3,1,1,2
6,0,0,1,2,0


In [4]:
df.describe()

,A,B,C,D,E
count,7.000000,7.000000,7.000000,7.000000,7.000000
mean,1.142857,1.428571,0.571429,1.714286,1.142857
std,1.951800,1.272418,0.534522,0.487950,1.573592
min,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.000000,0.500000,0.000000,1.500000,0.000000
50%,0.000000,1.000000,1.000000,2.000000,0.000000
75%,2.000000,2.500000,1.000000,2.000000,2.000000
max,4.000000,3.000000,1.000000,2.000000,4.000000


We create a df2 with a "test_col" column that will include all other columns.

In [5]:

df3 = df.copy()
df3['test_col'] = range(0, len(df))
df3 = df3.drop('E', 1)
df3

,A,B,C,D,test_col
0,0,0,0,2,0
1,0,1,0,2,1
2,0,2,0,2,2
3,0,3,1,2,3
4,4,1,1,1,4
5,4,3,1,1,5
6,0,0,1,2,6


For the naive solution, we generate the search space as follows :
all permutations of df columns against all permutations of df columns.

In [6]:
from itertools import permutations

column_combination = []
for i in range(1, len(df.columns)+1):
  column_combination.append(list(permutations(df.columns, i)))

instructions = []
for gen in column_combination:
  for permLeft in gen:
    for permRight in gen:
      #filter out trivial inclusions like A in A and AB in AB, etc...
      if (permLeft != permRight):
        instructions.append((permLeft, permRight))


instructions

[(('A',), ('B',)),
 (('A',), ('C',)),
 (('A',), ('D',)),
 (('A',), ('E',)),
 (('B',), ('A',)),
 (('B',), ('C',)),
 (('B',), ('D',)),
 (('B',), ('E',)),
 (('C',), ('A',)),
 (('C',), ('B',)),
 (('C',), ('D',)),
 (('C',), ('E',)),
 (('D',), ('A',)),
 (('D',), ('B',)),
 (('D',), ('C',)),
 (('D',), ('E',)),
 (('E',), ('A',)),
 (('E',), ('B',)),
 (('E',), ('C',)),
 (('E',), ('D',)),
 (('A', 'B'), ('A', 'C')),
 (('A', 'B'), ('A', 'D')),
 (('A', 'B'), ('A', 'E')),
 (('A', 'B'), ('B', 'A')),
 (('A', 'B'), ('B', 'C')),
 (('A', 'B'), ('B', 'D')),
 (('A', 'B'), ('B', 'E')),
 (('A', 'B'), ('C', 'A')),
 (('A', 'B'), ('C', 'B')),
 (('A', 'B'), ('C', 'D')),
 (('A', 'B'), ('C', 'E')),
 (('A', 'B'), ('D', 'A')),
 (('A', 'B'), ('D', 'B')),
 (('A', 'B'), ('D', 'C')),
 (('A', 'B'), ('D', 'E')),
 (('A', 'B'), ('E', 'A')),
 (('A', 'B'), ('E', 'B')),
 (('A', 'B'), ('E', 'C')),
 (('A', 'B'), ('E', 'D')),
 (('A', 'C'), ('A', 'B')),
 (('A', 'C'), ('A', 'D')),
 (('A', 'C'), ('A', 'E')),
 (('A', 'C'), ('B', 'A')),

We can see that the search space for 5 columns is 32 500 combinations (32 825 combinations if we don't remove the trivial inclusions) of column inclusions.

In [7]:
len(instructions)

32500

For concatenating all the columns into easily comparable tuples, we convert the dataframe values to string type :

In [8]:
df = df.applymap(str)
df

,A,B,C,D,E
0,0,0,0,2,0
1,0,1,0,2,0
2,0,2,0,2,2
3,0,3,1,2,0
4,4,1,1,1,4
5,4,3,1,1,2
6,0,0,1,2,0


A test of string concat between two columns :

In [9]:
df2 = df.copy()

df[['A', 'B']].agg(''.join, axis=1)

0    00
1    01
2    02
3    03
4    41
5    43
6    00
dtype: object

In [10]:
def concat_and_remove_duplicates(df, instruction):
  return df[list(instruction)].agg(''.join, axis=1).drop_duplicates()

concat_and_remove_duplicates(df, ('A', 'B', 'C', 'D'))

0    0002
1    0102
2    0202
3    0312
4    4111
5    4311
6    0012
dtype: object

In [11]:
#we then check how inclusions work :

test_df = concat_and_remove_duplicates(df, ('A', 'B', 'C', 'D'))

#0002 is included, 0003 is not. we want to negate these booleans to exclude exclusions(and get a list of exclusions)
print(not('0002' in test_df.values), not('0003' in test_df.values))

False True


For checking inclusion, we will concatenate all strings of a column combination, remove duplicates in resulting dataframes (the two dataframes created from the 2 combinations to check), and check for inclusion from the left df to the right df.

In [12]:
#test with 100 first instructions
instructions2 = instructions[:100]
list_of_exclusions = []

for instruction in instructions2:
  leftDF = concat_and_remove_duplicates(df, instruction[0])
  rightDF = concat_and_remove_duplicates(df, instruction[1])
  for leftRow in leftDF:
    if (not(leftRow in rightDF.values)):
      list_of_exclusions.append(instruction)
      break


# set(instructions2)
set(instructions2) - set(list_of_exclusions)

{(('A',), ('E',)),
 (('A', 'C'), ('A', 'B')),
 (('A', 'C'), ('E', 'B')),
 (('A', 'C'), ('E', 'C')),
 (('A', 'D'), ('A', 'B')),
 (('A', 'D'), ('E', 'D')),
 (('C',), ('B',)),
 (('D',), ('B',))}

The code underneath is used to display a progress bar in Google Colab by adding some HTML

In [13]:
from IPython.display import HTML, display
import time

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))
# #example    
out = display(progress(0, 100), display_id=True)

We can then sum up all this code into a single IND (inclusion depedency) function :

In [14]:
def ind(df):
  df = df.applymap(str)
  column_combination = []
  for i in range(1, len(df.columns)+1):
    column_combination.append(list(permutations(df.columns, i)))

  instructions = []
  for gen in column_combination:
    for permLeft in gen:
      for permRight in gen:
        #filter out trivial inclusions like A in A and AB in AB, etc...
        if (permLeft != permRight):
          instructions.append((permLeft, permRight))

  list_of_exclusions = []

  cpt = 0
  for instruction in instructions:
    leftDF = df[list(instruction[0])].agg(''.join, axis=1).drop_duplicates()
    rightDF = df[list(instruction[1])].agg(''.join, axis=1).drop_duplicates()
    for leftRow in leftDF:
      if (not(leftRow in rightDF.values)):
        list_of_exclusions.append(instruction)
        break
    cpt+=1
    
    #display bar
    out.update(progress(cpt, len(instructions)))

  #return all inclusions except the exclusions
  return set(instructions) - set(list_of_exclusions)

We run the function on the default dataset (>2 mins) : 

In [15]:
# ind(df)

We run the function on df2 to check how inclusions changes when we have a column that contains all values :

In [16]:
# ind(df3)

We find that all single columns are included in the test column and even find supersets of the test_column

In [17]:
# attributes from :
# https://archive.ics.uci.edu/ml/datasets/Forest+Fires

  #  1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
  #  2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
  #  3. month - month of the year: "jan" to "dec" 
  #  4. day - day of the week: "mon" to "sun"
  #  5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
  #  6. DMC - DMC index from the FWI system: 1.1 to 291.3 
  #  7. DC - DC index from the FWI system: 7.9 to 860.6 
  #  8. ISI - ISI index from the FWI system: 0.0 to 56.10
  #  9. temp - temperature in Celsius degrees: 2.2 to 33.30
  #  10. RH - relative humidity in %: 15.0 to 100
  #  11. wind - wind speed in km/h: 0.40 to 9.40 
  #  12. rain - outside rain in mm/m2 : 0.0 to 6.4 
  #  13. area - the burned area of the forest (in ha): 0.00 to 1090.84 
  #  (this output variable is very skewed towards 0.0, thus it may make
  #   sense to model with the logarithm transform). 

forest_fires = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv',
                        sep=',', encoding='latin-1')

forest_fires

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44
513,2,4,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29
514,7,4,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16
515,1,4,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00


In [20]:
forest_fires.loc[:, :'FFMC']

,X,Y,month,day,FFMC
0,7,5,mar,fri,86.2
1,7,4,oct,tue,90.6
2,7,4,oct,sat,90.6
3,8,6,mar,fri,91.7
4,8,6,mar,sun,89.3
...,...,...,...,...,...
512,4,3,aug,sun,81.6
513,2,4,aug,sun,81.6
514,7,4,aug,sun,81.6
515,1,4,aug,sat,94.4


In [21]:
# ind(forest_fires)
ind(forest_fires.loc[:, :'FFMC'])


{(('Y',), ('X',)), (('Y', 'day'), ('X', 'day')), (('day', 'Y'), ('day', 'X'))}